# Dimensionality Reduction

In this notebook we would talk about the dimensionality reduction techniques.

As the data gets more complex, it usually tends to have higher number of features (or dimensions) to it. So, it is desirable to reduce them. 

# Why Dimensionality Reduction?

* Helps reduce the dimensionality curse
* Space required to store the data is reduced as the number of dimensions comes down
* Less dimensions lead to less computation/training time
* Some algorithms do not perform well when we have a large dimensions. So reducing these dimensions needs to happen for the algorithm to be useful
* It takes care of multicollinearity by removing redundant features. For example, you have two variables – ‘time spent on treadmill in minutes’ and ‘calories burnt’. These variables are highly correlated as the more time you spend running on a treadmill, the more calories you will burn. Hence, there is no point in storing both as just one of them does what you require
* It helps in visualizing data. As discussed earlier, it is very difficult to visualize data in higher dimensions so reducing our space to 2D or 3D may allow us to plot and observe patterns more clearly

# Types of Dimensionality Reduction

There are mainly two types of dimensionality reduction methods. Both methods reduce the number of dimensions but in different ways. It is very important to distinguish between those two types of methods. 

One type of method only keeps the most important features in the dataset and removes the redundant features. There is no transformation applied to the set of features. Backward elimination, Forward selection and Random forests are examples of this method. 

The other method finds a combination of new features. An appropriate transformation is applied to the set of features. The new set of features contains different values instead of the original values. This method can be further divided into Linear methods and Non-linear methods. Non-linear methods are well known as Manifold learning. Principal Component Analysis (PCA), Factor Analysis (FA), Linear Discriminant Analysis (LDA) and Truncated Singular Value Decomposition (SVD) are examples of linear dimensionality reduction methods. Kernel PCA, t-distributed Stochastic Neighbor Embedding (t-SNE), Multidimensional Scaling (MDS) and Isometric mapping (Isomap) are examples of non-linear dimensionality reduction methods.

Now, let's start discussing some of the methods.

# Missing Value Ratio

Suppose you’re given a dataset. What would be your first step? You would naturally want to explore the data first before building model. While exploring the data, you find that your dataset has some missing values. Now what? You will try to find out the reason for these missing values and then impute them or drop the variables entirely which have missing values (using appropriate methods).

What if we have too many missing values (say more than 50%)? Should we impute the missing values or drop the variable? I would prefer to drop the variable since it will not have much information. However, this isn’t set in stone. We can set a threshold value and if the percentage of missing values in any variable is more than that threshold, we will drop the variable. This is usually powered by the domain knowledge.

# Low Variance Filter

Consider a variable in our dataset where all the observations have the same value, say 1. If we use this variable, do you think it can improve the model we will build? The answer is no, because this variable will have zero variance.

So, we need to calculate the variance of each variable we are given. Then drop the variables having low variance as compared to other variables in our dataset. The reason for doing this, as I mentioned above, is that variables with a low variance will not affect the target variable.

Do take care for categorical variables though. They would most likely need more attention and you should use counts of the unique values and other metrics for them to determine if the variable is adding any useful value or not. 

# High Correlation filter

High correlation between two variables means they have similar trends and are likely to carry similar information. This can bring down the performance of some models drastically (linear and logistic regression models, for instance). We can calculate the correlation between independent numerical variables that are numerical in nature. If the correlation coefficient crosses a certain threshold value, we can drop one of the variables (dropping a variable is highly subjective and should always be done keeping the domain in mind).

As a general guideline, we should keep those variables which show a decent or high correlation with the target variable.

# Random Forest 

Random Forest is one of the most widely used algorithms for feature selection. It comes packaged with in-built feature importance so you don’t need to program that separately. This helps us select a smaller subset of features. 

Although it's a great way to select features, beware of some important things you need to be wary of:

* **They are completely useless if your model is weak.** If your model does not generalize to validation data - like in the case you mentioned of completely random predictors - then feature importances have no meaning. That is because all the splits are simply overfitting the training data and not capturing any real trend, so all the gini impurity you sum is useless

* **They are strongly influenced by correlated features.** It is a fact. Just know it and perform some good old feature engineering before hand to avoid having features that are too correlated

* **They are biased towards numerical and high cardinality features.** This is definitely a problem. There are some alternative approaches to help relieve this

However, given that you've kept those things in mind, it can be a great way to select features.

Here's an example:

```python
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=1, max_depth=10)
df=pd.get_dummies(df)
model.fit(df, train.target_variable)
features = df.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-9:]  # top 10 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()
```

# Permutation Importance

Permutation feature importance is a model inspection technique that can be used for any fitted estimator when the data is tabular. This is especially useful for non-linear or opaque estimators. The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. This technique benefits from being model agnostic and can be calculated many times with different permutations of the feature.

**Important:** Features that are deemed of **low importance for a bad model** (low cross-validation score) could be **very important for a good model.** Therefore it is always important to evaluate the predictive power of a model using a held-out set (or better with cross-validation) prior to computing importances. Permutation importance does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model.

Here's an example:

```python
>>> clf = LogisticRegression().fit(X, y)
>>> result = permutation_importance(clf, X, y, n_repeats=10,
...                                 random_state=0)
>>> result.importances_mean
array([0.4666..., 0.       , 0.       ])
```

# Backward Feature Elimination

Follow the below steps to understand and use the ‘Backward Feature Elimination’ technique:

* We first take all the n variables present in our dataset and train the model using them
* We then calculate the performance of the model
* Now, we compute the performance of the model after eliminating each variable (n times), i.e., we drop one variable every time and train the model on the remaining n-1 variables
* We identify the variable whose removal has produced the smallest (or no) change in the performance of the model, and then drop that variable
* Repeat this process until no variable can be dropped

**Important:** Just like the Permutation Importance, features deemed **low importance for bad model** could be **very important for good model.** Therefore it is always important to evaluate the predictive power of a model using a held-out set (or better with cross-validation) prior to computing importances. This method does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model.

Also, it tends to be slow since a model is fit multiple times. You need to take training time and stuff into account.

Here's an example:

```python
>>> estimator = SVR(kernel="linear")
>>> selector = RFE(estimator, n_features_to_select=5, step=1)
>>> selector = selector.fit(X, y)
>>> selector.support_
array([ True,  True,  True,  True,  True, False, False, False, False,
       False])
>>> selector.ranking_
array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])
```

# Forward Feature Selection

This is the opposite process of the Backward Feature Elimination we saw above. Instead of eliminating features, we try to find the best features which improve the performance of the model. This technique works as follows:

* We start with a single feature. Essentially, we train the model n number of times using each feature separately
* The variable giving the best performance is selected as the starting variable
* Then we repeat this process and add one variable at a time. The variable that produces the highest increase in performance is retained
* We repeat this process until no significant improvement is seen in the model’s performance

**Important:** Just like the Permutation Importance, features deemed **low importance for bad model** could be **very important for good model.** Therefore it is always important to evaluate the predictive power of a model using a held-out set (or better with cross-validation) prior to computing importances. This method does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model.

Also, it tends to be slow since a model is fit multiple times. You need to take training time and stuff into account.

Here's an example:

```python
>>> knn = KNeighborsClassifier(n_neighbors=3)
>>> sfs = SequentialFeatureSelector(knn, n_features_to_select=3)
>>> sfs.fit(X, y)
SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3),
                          n_features_to_select=3)
>>> sfs.get_support()
array([ True, False,  True,  True])
>>> sfs.transform(X).shape  # .transform reduces the X
(150, 3)
```

# Principal Component Analysis or PCA

PCA is a technique which helps us in extracting a new set of variables from an existing large set of variables. These newly extracted variables are called Principal Components. You can refer to this article to learn more about PCA. For your quick reference, below are some of the key points you should know about PCA before proceeding further:

* A principal component is a linear combination of the original variables
* Principal components are extracted in such a way that the first principal component explains maximum variance in the dataset
* Second principal component tries to explain the remaining variance in the dataset and is uncorrelated to the first principal component
* Third principal component tries to explain the variance which is not explained by the first two principal components and so on

## Important Things to Consider

* Make sure to **standardize the variables** before running PCA since it's very sensitive to variance within the components.
* Do **not use on categorical variables** since the distance metric used for it won't make sense on categorical variables. 
* PCA makes the reduced data you get **not really interpretable** compared to the ones you have before.
* PCA is still quite **susceptible to throwing strong classification signals away**. PCA geometrically project a data set onto fewer dimensions, where the new variables are called principal components. This is done in such a way that the principal components are orthogonal and have the largest possible variances. So, if the variable is such that above the line it's one class (target variable) and below another, PCA would give up the result. Here's an example: <img src='https://miro.medium.com/max/875/1*F9jEYN5okmKNxjsodQgIow.png'/><br> Although this doesn't happen that much in real life.
* If variables are correlated non-linearly, you would lose them.

## Example

Here's an example:
```python
>>> pca = PCA(n_components=2)
>>> pca.fit(X)
PCA(n_components=2)
>>> print(pca.explained_variance_ratio_)
[0.9924... 0.0075...]
```

# Manifold Learning: Isomap

While PCA is a great dimensionality reduction algorithm, it only takes linear components. This would cause it to lose any non-linear relationships between variables. A set of techniques that take those things into account are called Manifold Learning. And Isomap is one of the standard ones. 

It uses the geodesic distance between two points:

<img src="https://miro.medium.com/max/716/1*txQ3bGq0SsFgWnxpjgZsaw.png"/>

It is a manifold learning algorithm which tries to preserve the geodesic distance between samples while reducing the dimension. It computes them by finding the neighbours of the distance. 

**Limitations:** Although it captures a lot more information, it has **problems with manifolds with holes.** Also it is **computationally expensive.**

Here's an example: 

```python
>>> X, _ = load_digits(return_X_y=True)
>>> X.shape
(1797, 64)
>>> embedding = Isomap(n_components=2, n_jobs=-1)  # n_jobs=-1 takes all cores to speed it up
>>> X_transformed = embedding.fit_transform(X[:100])
>>> X_transformed.shape
(100, 2)
``` 